In [2]:
import boto3
import sagemaker
import pandas as pd

import json

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [3]:
#pip install awswrangler
#%pip install boto3

In [4]:
import json
import boto3

s3 = boto3.resource('s3')
bucket =  'redditoutput'

#comment 
key1 = 'comment-2022-12-10-10-38.json'
obj1 = s3.Object(bucket, key1)
data1 = json.load(obj1.get()['Body']) 
type(data1)

key2 = 'submission-2022-12-11-01-52.json'
obj2 = s3.Object(bucket, key2) 
data2 = json.load(obj2.get()['Body']) 

In [5]:
from io import StringIO

comment_df = pd.read_json(StringIO(data1))
comment_df.head(2)

,com_parent_id,com_permalink,com_body,com_date,com_author,com_score
0,t1_izmjtkb,/r/wallstreetbets/comments/zhc398/breaking_egy...,India and China are at odds for a long time. T...,2022-12n-10,mzak88,1
1,t3_zhonq6,/r/wallstreetbets/comments/zhonq6/wsb_spank_ba...,Maybe that is the post awarding you an avatar!...,2022-12n-10,Lillica_Golden_SHIB,1


In [6]:
submission_df = pd.read_json(StringIO(data2)) 
submission_df.head(2)

,post_score,post_author,post_id,post_permalink,post_date,post_body
0,1,MIA3D,zi9xvw,/r/wallstreetbets/comments/zi9xvw/lost_everyth...,2022-12-11,Lost my entire life savings of 30k last year t...
1,1,New-Difference9684,zi8x4d,/r/wallstreetbets/comments/zi8x4d/elon_musk_tw...,2022-12-11,"In an unprecedented move, Tesla Motors CEO, an..."


In [7]:
com_index = ['com_score', 'com_permalink', 'com_author', 'com_parent_id', 'com_date', 'com_body'] 
sub_index = ['post_score', 'post_author', 'post_id', 'post_permalink', 'post_date', 'post_body']

In [8]:
# UNRELATED - just checking the S3 bucket 
import boto3

# Create an S3 client
s3 = boto3.client('s3')

# List the objects in your bucket
objects = s3.list_objects(Bucket='redditoutput')

# Loop through the objects in your bucket
for obj in objects['Contents']:
    # Print the object key
    print(obj['Key'])

comment-2022-12-10-10-38.json
comment-2022-12-11-01-52.json
comment-2022-12-11-14-37.json
comment-2022-12-12-04-16.json
submission-2022-12-10-10-38.json
submission-2022-12-11-01-52.json
submission-2022-12-11-14-37.json
submission-2022-12-12-04-15.json


## Comment df

In [9]:
import boto3

# Set your AWS access credentials
#AWS_ACCESS_KEY_ID = 'your_access_key'
#AWS_SECRET_ACCESS_KEY = 'your_secret_key'

# Set the name of the S3 bucket
bucket_name = 'redditoutput'

# Connect to the S3 bucket
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)

# Dictionary to store the content of each file 
files = {}

# Loop over the objects (files) in the bucket
for obj in bucket.objects.filter(Prefix="comment"):
    # Get the object key (filename)
    key=obj.key
    # Get the object content (file content)
    content = json.load(obj.get()['Body'])#.read().decode('ascii'))
    files[key] = content

In [10]:
files.keys()

dict_keys(['comment-2022-12-10-10-38.json', 'comment-2022-12-11-01-52.json', 'comment-2022-12-11-14-37.json', 'comment-2022-12-12-04-16.json'])

In [11]:
#comment 1 
com = files['comment-2022-12-10-10-38.json']
from io import StringIO
com_df = pd.read_json(StringIO(com))
com_df.head()

,com_parent_id,com_permalink,com_body,com_date,com_author,com_score
0,t1_izmjtkb,/r/wallstreetbets/comments/zhc398/breaking_egy...,India and China are at odds for a long time. T...,2022-12n-10,mzak88,1
1,t3_zhonq6,/r/wallstreetbets/comments/zhonq6/wsb_spank_ba...,Maybe that is the post awarding you an avatar!...,2022-12n-10,Lillica_Golden_SHIB,1
2,t1_izn1z5z,/r/wallstreetbets/comments/zh9l7a/hey_how_to_f...,Market is getting connected to reality again,2022-12n-10,Imaginary-Jaguar662,1
3,t1_izn45jh,/r/wallstreetbets/comments/zhc398/breaking_egy...,Remind me in 30 years.,2022-12n-10,THE-Sumukh,1
4,t1_izn80yc,/r/wallstreetbets/comments/zhc398/breaking_egy...,Found the bhakt.,2022-12n-10,SalmonCanSwimToJapan,1


In [12]:
#comment 12-11 / 14.37 
com2 = files['comment-2022-12-11-14-37.json']
from io import StringIO
com_df2 = pd.read_json(StringIO(com2))
com_df2.count()

com_parent_id    1748
com_permalink    1748
com_body         1748
com_date         1748
com_author       1748
com_score        1748
dtype: int64

In [13]:
# our dataframe baby is here 
com_result = pd.concat([com_df, com_df2], ignore_index=True, sort=False)
com_result

,com_parent_id,com_permalink,com_body,com_date,com_author,com_score
0,t1_izmjtkb,/r/wallstreetbets/comments/zhc398/breaking_egy...,India and China are at odds for a long time. T...,2022-12n-10,mzak88,1
1,t3_zhonq6,/r/wallstreetbets/comments/zhonq6/wsb_spank_ba...,Maybe that is the post awarding you an avatar!...,2022-12n-10,Lillica_Golden_SHIB,1
2,t1_izn1z5z,/r/wallstreetbets/comments/zh9l7a/hey_how_to_f...,Market is getting connected to reality again,2022-12n-10,Imaginary-Jaguar662,1
3,t1_izn45jh,/r/wallstreetbets/comments/zhc398/breaking_egy...,Remind me in 30 years.,2022-12n-10,THE-Sumukh,1
4,t1_izn80yc,/r/wallstreetbets/comments/zhc398/breaking_egy...,Found the bhakt.,2022-12n-10,SalmonCanSwimToJapan,1
...,...,...,...,...,...,...
1993,t1_izr1acf,/r/wallstreetbets/comments/zi9xvw/lost_everyth...,Divorce is cheap when you have no assets to yo...,2022-12-11,VaccinatedZombie,1
1994,t3_zh8kto,/r/wallstreetbets/comments/zh8kto/weekend_disc...,Milfs r great. But sometimes they have problem...,2022-12-11,plu5on3,1
1995,t3_zi9xvw,/r/wallstreetbets/comments/zi9xvw/lost_everyth...,Lmao dude needs to prioritize his life. You fu...,2022-12-11,Sealbeater,1
1996,t1_izr5rqw,/r/wallstreetbets/comments/zi4zbo/companies_wa...,Well laid out. Everyone is trying to bail on t...,2022-12-11,euphoricgazer,1


## Submission df 

In [14]:
import boto3

# Set your AWS access credentials
#AWS_ACCESS_KEY_ID = 'your_access_key'
#AWS_SECRET_ACCESS_KEY = 'your_secret_key'

# Set the name of the S3 bucket
bucket_name = 'redditoutput'

# Connect to the S3 bucket
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)

# Dictionary to store the content of each file 
files2 = {}

# Loop over the objects (files) in the bucket
for obj in bucket.objects.filter(Prefix="submission"):
    # Get the object key (filename)
    key=obj.key
    # Get the object content (file content)
    content = json.load(obj.get()['Body'])#.read().decode('ascii'))
    files2[key] = content

In [15]:
files2.keys()

dict_keys(['submission-2022-12-10-10-38.json', 'submission-2022-12-11-01-52.json', 'submission-2022-12-11-14-37.json', 'submission-2022-12-12-04-15.json'])

In [16]:
#sub 1 
sub = files2['submission-2022-12-10-10-38.json']
from io import StringIO
sub_df = pd.read_json(StringIO(sub))
sub_df.count()

post_score        32
post_author       32
post_id           32
post_permalink    32
post_date         32
post_body         32
dtype: int64

In [17]:
#sub 2 
sub2 = files2['submission-2022-12-11-01-52.json']
from io import StringIO
sub_df2 = pd.read_json(StringIO(sub2))
sub_df2.count()

post_score        69
post_author       69
post_id           69
post_permalink    69
post_date         69
post_body         69
dtype: int64

In [18]:
#sub 3 
sub3 = files2['submission-2022-12-11-14-37.json']
from io import StringIO
sub_df3 = pd.read_json(StringIO(sub3))
sub_df3.count()

post_score        0
post_author       0
post_id           0
post_permalink    0
post_date         0
post_body         0
dtype: int64

In [19]:
# our dataframe baby is here 
sub_result = pd.concat([sub_df, sub_df2,  sub_df3], ignore_index=True, sort=False)
sub_result

,post_score,post_author,post_id,post_permalink,post_date,post_body
0,1.0,inegyio,zhp8qs,/r/wallstreetbets/comments/zhp8qs/these_are_th...,2022-12-10,I have tested several thousand strategies and ...
1,1.0,CandidPhrase3058,zhp469,/r/wallstreetbets/comments/zhp469/polestarpsny...,2022-12-10,PSNY WILL hit 50$ soon. \n\nVery undervalued \...
2,1.0,kingsniger9019,zhoo9k,/r/wallstreetbets/comments/zhoo9k/golden_nugget/,2022-12-10,[removed]
3,1.0,Rekeesyenom,zhohjg,/r/wallstreetbets/comments/zhohjg/was_wonderin...,2022-12-10,[removed]
4,1.0,Shakavon,zhob9s,/r/wallstreetbets/comments/zhob9s/unlock_the_p...,2022-12-10,\n\n\nhttps://preview.redd.it/dm2n1937k15a1....
...,...,...,...,...,...,...
96,1.0,Ok-Tangerine-7196,zhz7ih,/r/wallstreetbets/comments/zhz7ih/the_lizzo_ef...,2022-12-10,[The Lizzo Effect](https://youtu.be/1bGG1IRacR...
97,1.0,stUS95,zhz4a8,/r/wallstreetbets/comments/zhz4a8/top_stocks_f...,2022-12-10,[removed]
98,1.0,jdd91500,zhywqt,/r/wallstreetbets/comments/zhywqt/the_12_phras...,2022-12-10,\nEpisode 1: he is introduced to SBF and learn...
99,1.0,DrunkenRedittor,zhytyy,/r/wallstreetbets/comments/zhytyy/i_asked_a_yo...,2022-12-10,His answer was :\n\n&gt; VIX moves opposite to...


## 2. Data cleaning 

- we have 2 datasets: sub_result (submission.json) and com_result (comments.json)

#### 2.1 cleaning n out of the dataframe for comment results 

In [20]:
wht = com_result[com_result['com_date'].str.contains("n")]


# Replace the character "n" with an empty string
wht["com_date"] = wht["com_date"].str.replace("n", "")

# Print the resulting DataFrame
wht

com_result =com_result.drop(com_result[com_result["com_date"].str.contains("n")].index)
com_result = pd.concat([com_result,wht], ignore_index=True, sort=False)
com_result[com_result['com_date'].str.contains("n")].any()

/tmp/ipykernel_2520/1323327534.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wht["com_date"] = wht["com_date"].str.replace("n", "")


com_parent_id    False
com_permalink    False
com_body         False
com_date         False
com_author       False
com_score        False
dtype: bool

In [21]:
com_result.head(2)

,com_parent_id,com_permalink,com_body,com_date,com_author,com_score
0,t3_zi9xvw,/r/wallstreetbets/comments/zi9xvw/lost_everyth...,"Stop gambling, this is not how to get rich. Da...",2022-12-11,ScientistFew6563,1
1,t3_zi9xvw,/r/wallstreetbets/comments/zi9xvw/lost_everyth...,There is no way you saved up $30k on a $36k in...,2022-12-11,Freebandz1,1


In [22]:
sub_result.head(2)

,post_score,post_author,post_id,post_permalink,post_date,post_body
0,1.0,inegyio,zhp8qs,/r/wallstreetbets/comments/zhp8qs/these_are_th...,2022-12-10,I have tested several thousand strategies and ...
1,1.0,CandidPhrase3058,zhp469,/r/wallstreetbets/comments/zhp469/polestarpsny...,2022-12-10,PSNY WILL hit 50$ soon. \n\nVery undervalued \...


### 2.2 cleaning duplicates 

In [23]:
com_result['com_permalink'].duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
1993    False
1994    False
1995    False
1996    False
1997    False
Name: com_permalink, Length: 1998, dtype: bool

In [24]:
duplicates = com_result.duplicated()
com_result = com_result[~duplicates]
com_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1998 entries, 0 to 1997
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   com_parent_id  1998 non-null   object
 1   com_permalink  1998 non-null   object
 2   com_body       1998 non-null   object
 3   com_date       1998 non-null   object
 4   com_author     1998 non-null   object
 5   com_score      1998 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 109.3+ KB


In [25]:
duplicates2 = sub_result.duplicated()
sub_result = sub_result[~duplicates2]
sub_result

,post_score,post_author,post_id,post_permalink,post_date,post_body
0,1.0,inegyio,zhp8qs,/r/wallstreetbets/comments/zhp8qs/these_are_th...,2022-12-10,I have tested several thousand strategies and ...
1,1.0,CandidPhrase3058,zhp469,/r/wallstreetbets/comments/zhp469/polestarpsny...,2022-12-10,PSNY WILL hit 50$ soon. \n\nVery undervalued \...
2,1.0,kingsniger9019,zhoo9k,/r/wallstreetbets/comments/zhoo9k/golden_nugget/,2022-12-10,[removed]
3,1.0,Rekeesyenom,zhohjg,/r/wallstreetbets/comments/zhohjg/was_wonderin...,2022-12-10,[removed]
4,1.0,Shakavon,zhob9s,/r/wallstreetbets/comments/zhob9s/unlock_the_p...,2022-12-10,\n\n\nhttps://preview.redd.it/dm2n1937k15a1....
...,...,...,...,...,...,...
96,1.0,Ok-Tangerine-7196,zhz7ih,/r/wallstreetbets/comments/zhz7ih/the_lizzo_ef...,2022-12-10,[The Lizzo Effect](https://youtu.be/1bGG1IRacR...
97,1.0,stUS95,zhz4a8,/r/wallstreetbets/comments/zhz4a8/top_stocks_f...,2022-12-10,[removed]
98,1.0,jdd91500,zhywqt,/r/wallstreetbets/comments/zhywqt/the_12_phras...,2022-12-10,\nEpisode 1: he is introduced to SBF and learn...
99,1.0,DrunkenRedittor,zhytyy,/r/wallstreetbets/comments/zhytyy/i_asked_a_yo...,2022-12-10,His answer was :\n\n&gt; VIX moves opposite to...


## 3. extracting tickers 

In [26]:
#pip install reticker
import reticker

extractor = reticker.TickerExtractor()
type(extractor.pattern)

re.Pattern

In [27]:
com_result.dtypes

com_parent_id    object
com_permalink    object
com_body         object
com_date         object
com_author       object
com_score         int64
dtype: object

### extract comment tickers

In [28]:
com_result['ticker'] = com_result['com_body'].apply(reticker.TickerExtractor().extract)
com_result.head(2)

,com_parent_id,com_permalink,com_body,com_date,com_author,com_score,ticker
0,t3_zi9xvw,/r/wallstreetbets/comments/zi9xvw/lost_everyth...,"Stop gambling, this is not how to get rich. Da...",2022-12-11,ScientistFew6563,1,[]
1,t3_zi9xvw,/r/wallstreetbets/comments/zi9xvw/lost_everyth...,There is no way you saved up $30k on a $36k in...,2022-12-11,Freebandz1,1,[]


### extract submission tickers

In [29]:
sub_result['ticker'] = sub_result['post_body'].apply(reticker.TickerExtractor().extract)
sub_result.head()

,post_score,post_author,post_id,post_permalink,post_date,post_body,ticker
0,1.0,inegyio,zhp8qs,/r/wallstreetbets/comments/zhp8qs/these_are_th...,2022-12-10,I have tested several thousand strategies and ...,"[TSLA, NY]"
1,1.0,CandidPhrase3058,zhp469,/r/wallstreetbets/comments/zhp469/polestarpsny...,2022-12-10,PSNY WILL hit 50$ soon. \n\nVery undervalued \...,"[PSNY, WILL]"
2,1.0,kingsniger9019,zhoo9k,/r/wallstreetbets/comments/zhoo9k/golden_nugget/,2022-12-10,[removed],[]
3,1.0,Rekeesyenom,zhohjg,/r/wallstreetbets/comments/zhohjg/was_wonderin...,2022-12-10,[removed],[]
4,1.0,Shakavon,zhob9s,/r/wallstreetbets/comments/zhob9s/unlock_the_p...,2022-12-10,\n\n\nhttps://preview.redd.it/dm2n1937k15a1....,"[RSI, GDP]"


## 4. Group by tickers 

In [30]:
#pip install -U textblob

In [31]:
com_result.dtypes

com_parent_id    object
com_permalink    object
com_body         object
com_date         object
com_author       object
com_score         int64
ticker           object
dtype: object

In [32]:
com_result['com_body']= com_result['com_body'].astype('string')
com_result.dtypes

com_parent_id    object
com_permalink    object
com_body         string
com_date         object
com_author       object
com_score         int64
ticker           object
dtype: object

# 5. NLP 
- For each group of stock prices, use a natural language processing (NLP) tool or library to perform sentiment analysis on the text associated with the ticker symbol. This could include news articles, company announcements, and other relevant text data.

In [33]:
import nltk 
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from nltk.tokenize import word_tokenize, RegexpTokenizer 
from nltk.corpus import stopwords 

import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams['figure.figsize'] = (10,8)
import seaborn as sns 
sns.set(style="whitegrid", palette = "Dark2") 
from wordcloud import WordCloud 

In [34]:
nltk.download('vader_lexicon') 
nltk.download('punkt') #tokenizer
nltk.download('stopwords') 

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
com_result.columns

Index(['com_parent_id', 'com_permalink', 'com_body', 'com_date', 'com_author',
       'com_score', 'ticker'],
      dtype='object')

In [36]:
#com_result['com_body'] = com_result['com_body'].astype('string')
com_result.dtypes
#pip install spacy 

com_parent_id    object
com_permalink    object
com_body         string
com_date         object
com_author       object
com_score         int64
ticker           object
dtype: object

## 5.1 Formatting for sentiment analysis  

### format for comments 

In [37]:
#tokenize 
com_result['new_reviews'] = com_result.apply(lambda row: nltk.word_tokenize(row['com_body']), axis=1)

#lowercase 
com_result['new_reviews'] = com_result['com_body'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#remove punc 
com_result['new_reviews'] = com_result['new_reviews'].str.replace('[^\w\s]','')

#stopwords 
stop = stopwords.words('english')
com_result['new_reviews'] = com_result['new_reviews'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

/tmp/ipykernel_2520/3639758241.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  com_result['new_reviews'] = com_result['new_reviews'].str.replace('[^\w\s]','')


In [38]:
com_result.head(2)
#dying but it's ok

,com_parent_id,com_permalink,com_body,com_date,com_author,com_score,ticker,new_reviews
0,t3_zi9xvw,/r/wallstreetbets/comments/zi9xvw/lost_everyth...,"Stop gambling, this is not how to get rich. Da...",2022-12-11,ScientistFew6563,1,[],stop gambling get rich damage life
1,t3_zi9xvw,/r/wallstreetbets/comments/zi9xvw/lost_everyth...,There is no way you saved up $30k on a $36k in...,2022-12-11,Freebandz1,1,[],way saved 30k 36k income 1800 monthly payments...


# 5.2 Vader Sentiment Analysis 
- Passing data['Text'] through pandas.apply() function and then using SentimentIntensityAnalyzer() does the trick. The fact is the function is not capable of handling the series object. Passing one by one element to SentimentIntensityAnalyzer() using a looping function does the job in this case.

In [39]:
com_result['new_reviews']=com_result['new_reviews'].apply(lambda x: x.encode("UTF-8").decode("ascii","ignore"))

### format for submissions 

In [40]:
#tokenize 
sub_result['new_reviews'] = sub_result.apply(lambda row: nltk.word_tokenize(row['post_body']), axis=1)

#lowercase 
sub_result['new_reviews'] = sub_result['post_body'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#remove punc 
sub_result['new_reviews'] = sub_result['new_reviews'].str.replace('[^\w\s]','')

#stopwords 
stop = stopwords.words('english')
sub_result['new_reviews'] = sub_result['new_reviews'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

/tmp/ipykernel_2520/2162903551.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  sub_result['new_reviews'] = sub_result['new_reviews'].str.replace('[^\w\s]','')


In [41]:
sub_result.head(2)

,post_score,post_author,post_id,post_permalink,post_date,post_body,ticker,new_reviews
0,1.0,inegyio,zhp8qs,/r/wallstreetbets/comments/zhp8qs/these_are_th...,2022-12-10,I have tested several thousand strategies and ...,"[TSLA, NY]",tested several thousand strategies variations ...
1,1.0,CandidPhrase3058,zhp469,/r/wallstreetbets/comments/zhp469/polestarpsny...,2022-12-10,PSNY WILL hit 50$ soon. \n\nVery undervalued \...,"[PSNY, WILL]",psny hit 50 soon undervalued short interest ri...


In [42]:
sub_result['new_reviews']=sub_result['new_reviews'].apply(lambda x: x.encode("UTF-8").decode("ascii","ignore"))

In [43]:
#for value in com_result['new_reviews']:
#value.encode()

In [44]:
#com_result= pd.Series(com_result['new_reviews'], dtype="string")
#com_result

In [45]:
#pip install vader 
#!pip install vaderSentiment

### Sentiment Analysis & Scores 

#### Comment 

In [46]:
from nltk import *
import pandas as pd
import vader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

comments = com_result

score_compound = []

for i in range(0, comments.shape[0]):
    
    score = analyzer.polarity_scores(comments.iloc[i][7])
    score1 = score['compound']
    score_compound.append(score1)
    
                
                
com_result['score_compound'] = score_compound
com_result.head()

,com_parent_id,com_permalink,com_body,com_date,com_author,com_score,ticker,new_reviews,score_compound
0,t3_zi9xvw,/r/wallstreetbets/comments/zi9xvw/lost_everyth...,"Stop gambling, this is not how to get rich. Da...",2022-12-11,ScientistFew6563,1,[],stop gambling get rich damage life,-0.2023
1,t3_zi9xvw,/r/wallstreetbets/comments/zi9xvw/lost_everyth...,There is no way you saved up $30k on a $36k in...,2022-12-11,Freebandz1,1,[],way saved 30k 36k income 1800 monthly payments...,0.0000
2,t3_zh8kto,/r/wallstreetbets/comments/zh8kto/weekend_disc...,LMAOOOOOO My training plan today is a hour an...,2022-12-11,TradeSpY420,1,[],lmaoooooo training plan today hour 20 minutes ...,0.3612
3,t1_izrqv4e,/r/wallstreetbets/comments/zi9xvw/lost_everyth...,One hard inquiry every 6 months isn't going to...,2022-12-11,uber765,1,[],one hard inquiry every 6 months isnt going fuc...,0.3807
4,t1_izseqeu,/r/wallstreetbets/comments/zh8kto/weekend_disc...,Why drag out the pain ![img](emote|t5_2th52|4260),2022-12-11,npcdad,1,[],drag pain imgemotet5_2th524260,-0.6369


#### Submisssion 


In [47]:
from nltk import *
import pandas as pd
import vader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

sub_comments = sub_result

score_compound2 = []

for i in range(0, sub_comments.shape[0]):
               score = analyzer.polarity_scores(sub_comments.iloc[i][7])
               score1 = score['compound']
               score_compound2.append(score1)
                
                

sub_result['score_compound'] = score_compound2
sub_result.head()

,post_score,post_author,post_id,post_permalink,post_date,post_body,ticker,new_reviews,score_compound
0,1.0,inegyio,zhp8qs,/r/wallstreetbets/comments/zhp8qs/these_are_th...,2022-12-10,I have tested several thousand strategies and ...,"[TSLA, NY]",tested several thousand strategies variations ...,0.5106
1,1.0,CandidPhrase3058,zhp469,/r/wallstreetbets/comments/zhp469/polestarpsny...,2022-12-10,PSNY WILL hit 50$ soon. \n\nVery undervalued \...,"[PSNY, WILL]",psny hit 50 soon undervalued short interest ri...,0.8689
2,1.0,kingsniger9019,zhoo9k,/r/wallstreetbets/comments/zhoo9k/golden_nugget/,2022-12-10,[removed],[],removed,0.0000
3,1.0,Rekeesyenom,zhohjg,/r/wallstreetbets/comments/zhohjg/was_wonderin...,2022-12-10,[removed],[],removed,0.0000
4,1.0,Shakavon,zhob9s,/r/wallstreetbets/comments/zhob9s/unlock_the_p...,2022-12-10,\n\n\nhttps://preview.redd.it/dm2n1937k15a1....,"[RSI, GDP]",httpspreviewredditdm2n1937k15a1jpgwidth512ampf...,0.9995


#### another method 

In [48]:
sid = SentimentIntensityAnalyzer()
pos_word_list=[]
neu_word_list=[]
neg_word_list=[]

for word in com_result['new_reviews']:
    if (sid.polarity_scores(word)['compound']) >= 0.1:
        pos_word_list.append(word)
    elif (sid.polarity_scores(word)['compound']) <= -0.1:
        neg_word_list.append(word)
    else:
        neu_word_list.append(word)        

In [49]:
#print('Positive:',pos_word_list)        
#print('Neutral:',neu_word_list)    
#print('Negative:',neg_word_list) 
score = sid.polarity_scores(com_result)
print('\nScores:', score)


Scores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


# 6. sentiment score 
- The NLP tool or library will output a sentiment score for each group of stock prices. This score can range from -1 (extremely negative sentiment) to 1 (extremely positive sentiment).

In [50]:
ini_list1 = sub_result['new_reviews']

ini_list2 = sub_result['ticker']

if isinstance(ini_list1, list):
  print("your object is a list !")
else:
    print("your object is not a list")
    
if isinstance(ini_list2, list):
    print("your object is a list")
else:
    print("your object is not a list")

your object is not a list
your object is not a list


In [51]:
#very important, we need it as tuple not list 
sub_result["ticker"] = sub_result["ticker"].map(tuple)

In [52]:
import pandas as pd


# use SentimentIntensityAnalyzer to analyze the text
sia = SentimentIntensityAnalyzer()
scores = sub_result['new_reviews'].apply(sia.polarity_scores)

# add the scores to the DataFrame
sub_result['scores'] = scores

# define a function to combine the scores for each group
def combine_scores(scores):
    # create an empty dictionary to store the combined scores
    combined_scores = {}

    # loop through each score and combine them
    for score in scores:
        for key in score:
            if key in combined_scores:
                combined_scores[key] += score[key]
            else:
                combined_scores[key] = score[key]

    # return the combined scores
    return combined_scores

# group the scores by ticker and calculate the combined score for each group
grouped_scores = sub_result.groupby('ticker')['scores']

# print the results
print(grouped_scores)


In [53]:
import pandas as pd


# use SentimentIntensityAnalyzer to analyze the text
sia = SentimentIntensityAnalyzer()
scores = com_result['new_reviews'].apply(sia.polarity_scores)

# add the scores to the DataFrame
com_result['scores'] = scores

# define a function to combine the scores for each group
def combine_scores(scores):
    # create an empty dictionary to store the combined scores
    combined_scores = {}

    # loop through each score and combine them
    for score in scores:
        for key in score:
            if key in combined_scores:
                combined_scores[key] += score[key]
            else:
                combined_scores[key] = score[key]

    # return the combined scores
    return combined_scores

# group the scores by ticker and calculate the combined score for each group
grouped_scores2 = com_result.groupby('ticker')['scores']

# print the results
print(grouped_scores2)


In [54]:
#for values in grouped_scores: 
#    print(values) 

In [55]:
sub_result.dropna(subset=['ticker'])

sub_result_new = sub_result[sub_result.ticker !=()]
sub_result_new.head(2)

,post_score,post_author,post_id,post_permalink,post_date,post_body,ticker,new_reviews,score_compound,scores
0,1.0,inegyio,zhp8qs,/r/wallstreetbets/comments/zhp8qs/these_are_th...,2022-12-10,I have tested several thousand strategies and ...,"(TSLA, NY)",tested several thousand strategies variations ...,0.5106,"{'neg': 0.0, 'neu': 0.958, 'pos': 0.042, 'comp..."
1,1.0,CandidPhrase3058,zhp469,/r/wallstreetbets/comments/zhp469/polestarpsny...,2022-12-10,PSNY WILL hit 50$ soon. \n\nVery undervalued \...,"(PSNY, WILL)",psny hit 50 soon undervalued short interest ri...,0.8689,"{'neg': 0.0, 'neu': 0.546, 'pos': 0.454, 'comp..."


In [72]:
sub_sorted_score = sub_result_new.sort_values('score_compound', ascending=False)
sub_sorted_score = sub_sorted_score.groupby('ticker')
sub_sorted_score.head()

,post_score,post_author,post_id,post_permalink,post_date,post_body,ticker,new_reviews,score_compound,scores
4,1.0,Shakavon,zhob9s,/r/wallstreetbets/comments/zhob9s/unlock_the_p...,2022-12-10,\n\n\nhttps://preview.redd.it/dm2n1937k15a1....,"(RSI, GDP)",httpspreviewredditdm2n1937k15a1jpgwidth512ampf...,0.9995,"{'neg': 0.079, 'neu': 0.671, 'pos': 0.25, 'com..."
37,1.0,chadsexytime,zi88z9,/r/wallstreetbets/comments/zi88z9/i_took_out_a...,2022-12-11,The beanie baby market boomed in the middle of...,"(MINT, PSA, TLDR)",beanie baby market boomed middle 90s older bea...,0.9938,"{'neg': 0.022, 'neu': 0.698, 'pos': 0.279, 'co..."
79,1.0,officialjd,zi0prm,/r/wallstreetbets/comments/zi0prm/i_took_out_a...,2022-12-10,The sports card market boomed in the middle of...,"(PSA, NBA)",sports card market boomed middle pandemic olde...,0.9876,"{'neg': 0.021, 'neu': 0.755, 'pos': 0.224, 'co..."
68,1.0,Minute-Remove-1672,zi29r7,/r/wallstreetbets/comments/zi29r7/i_dont_see_c...,2022-12-10,I have been paying attention to the post since...,"(US, NOC, BP, RIG, TIGR, BTU, NIO, BILI, NFLX,...",paying attention post since gamestop changed c...,0.9854,"{'neg': 0.117, 'neu': 0.619, 'pos': 0.264, 'co..."
67,1.0,Minute-Remove-1672,zi2hmw,/r/wallstreetbets/comments/zi2hmw/why_cant_i_s...,2022-12-10,I have been paying attention to the post since...,"(US, NOC, BP, RIG, TIGR, BTU, NIO, BILI, NFLX,...",paying attention post since gamestop changed c...,0.9854,"{'neg': 0.117, 'neu': 0.619, 'pos': 0.264, 'co..."
39,1.0,BeerorCoffee,zi81p2,/r/wallstreetbets/comments/zi81p2/i_took_out_a...,2022-12-11,The banana sticker market boomed in the middle...,"(PSA, TLDR)",banana sticker market boomed middle pandemic o...,0.9815,"{'neg': 0.042, 'neu': 0.719, 'pos': 0.239, 'co..."
34,1.0,kimillsonsdad,zi8tm0,/r/wallstreetbets/comments/zi8tm0/make_it_or_b...,2022-12-11,Next week seems like a swing trade type week a...,"(SQ,)",next week seems like swing trade type week bel...,0.9774,"{'neg': 0.069, 'neu': 0.622, 'pos': 0.309, 'co..."
83,1.0,CDNWSYOLOER,zi0lbr,/r/wallstreetbets/comments/zi0lbr/space_the_ne...,2022-12-10,Remember the tech boom in the 90s to early 200...,"(EV, RIGHT, NOW, SO, CLEAR, IN, FRONT, OF, YOU...",remember tech boom 90s early 2000s development...,0.9711,"{'neg': 0.064, 'neu': 0.787, 'pos': 0.149, 'co..."
76,1.0,The_Magnificent_007,zi0y1u,/r/wallstreetbets/comments/zi0y1u/redditwallst...,2022-12-10,I'm very surprise that no ETF has ever come ou...,"(JC, WSB-ETF)",im surprise etf ever come reddit one would thi...,0.9545,"{'neg': 0.032, 'neu': 0.648, 'pos': 0.32, 'com..."
58,1.0,MargotRobbieExplains,zi49de,/r/wallstreetbets/comments/zi49de/cron_cronos_...,2022-12-10,i would really love to hear your take on crono...,"(CRON, US)",would really love hear take cronos group stock...,0.9131,"{'neg': 0.0, 'neu': 0.701, 'pos': 0.299, 'comp..."


### comment

In [57]:
#very important, we need it as tuple not list 
com_result["ticker"] = com_result["ticker"].map(tuple)

In [58]:
com_result_new = com_result[com_result.ticker !=()]
com_result_new.head(2)

,com_parent_id,com_permalink,com_body,com_date,com_author,com_score,ticker,new_reviews,score_compound,scores
17,t3_zh8kto,/r/wallstreetbets/comments/zh8kto/weekend_disc...,NFT FOMO,2022-12-11,Parcero45,1,"(FOMO,)",nft fomo,0.0000,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
30,t1_izqhcb2,/r/wallstreetbets/comments/zh8kto/weekend_disc...,HOW do you equate my refusal to take a governm...,2022-12-11,TheseYoung6546,1,"(HOW,)",equate refusal take government sponsored exper...,0.4854,"{'neg': 0.128, 'neu': 0.59, 'pos': 0.283, 'com..."


In [70]:
com_sorted_score = com_result_new.sort_values('score_compound', ascending=False)

In [71]:
com_sorted_score = com_sorted_score.groupby('ticker')
com_sorted_score.head()

,com_parent_id,com_permalink,com_body,com_date,com_author,com_score,ticker,new_reviews,score_compound,scores
1740,t1_izqfobz,/r/wallstreetbets/comments/zi9xvw/lost_everyth...,This is solid advice and it sounds like you ar...,2022-12-11,StandardChaseScene,1,"(CA,)",solid advice sounds like ready takes lot peopl...,0.9923,"{'neg': 0.081, 'neu': 0.691, 'pos': 0.229, 'co..."
425,t1_izs43np,/r/wallstreetbets/comments/zhc398/breaking_egy...,&gt; Im not avoiding the question - the answer...,2022-12-11,voicesfromvents,1,"(US, ECB)",gt im avoiding question answer implicit acknow...,0.9806,"{'neg': 0.101, 'neu': 0.65, 'pos': 0.25, 'comp..."
862,t3_zi9xvw,/r/wallstreetbets/comments/zi9xvw/lost_everyth...,Its not too late to recover if you put your mi...,2022-12-11,S9CLAVE,1,"(US, VA, GOTO)",late recover put mind commit file bankruptcy d...,0.9709,"{'neg': 0.044, 'neu': 0.711, 'pos': 0.245, 'co..."
1472,t1_izrihqk,/r/wallstreetbets/comments/zhwbdy/2023_soft_la...,Yeah. Interesting story about 3407. I was pla...,2022-12-11,Benry26,1,"(TBD, TV)",yeah interesting story 3407 playing nintendo w...,0.9486,"{'neg': 0.104, 'neu': 0.691, 'pos': 0.205, 'co..."
183,t1_izs7eov,/r/wallstreetbets/comments/zi4zbo/companies_wa...,It's great we dropped AT&amp;T and I have foun...,2022-12-11,Skywalker0138,1,"(AT,)",great dropped atampt found channels saved 83 m...,0.9451,"{'neg': 0.0, 'neu': 0.481, 'pos': 0.519, 'comp..."
...,...,...,...,...,...,...,...,...,...,...
188,t3_zij46a,/r/wallstreetbets/comments/zij46a/what_do_you_...,WSB sucks now. I see these garbage posts in my...,2022-12-11,deTocqueville22,1,"(DD,)",wsb sucks see garbage posts feed time never us...,-0.8503,"{'neg': 0.402, 'neu': 0.538, 'pos': 0.06, 'com..."
682,t1_izmk0ok,/r/wallstreetbets/comments/zgyj72/2022_been_a_...,There isnt a real estate bubble in the sense t...,2022-12-11,mortgagemammoth,1,"(US, GDP, RE)",isnt real estate bubble sense theres going bur...,-0.8856,"{'neg': 0.202, 'neu': 0.687, 'pos': 0.111, 'co..."
359,t3_zh8kto,/r/wallstreetbets/comments/zh8kto/weekend_disc...,STOOPID FUCKS REALLY THOUGHT TWITTER WAS NOT G...,2022-12-11,LOVE2FUKWITHPP,1,"(FUCKS, REALLY, WAS, NOT, GONNA, WORK, AFTER, ...",stoopid fucks really thought twitter gonna wor...,-0.8885,"{'neg': 0.441, 'neu': 0.392, 'pos': 0.167, 'co..."
442,t3_zi9xvw,/r/wallstreetbets/comments/zi9xvw/lost_everyth...,"Honestly, if Im her and my SO has hidden the f...",2022-12-11,MichaelG_02,1,"(SO, OP)",honestly im hidden fact hes lost 30k savings m...,-0.9196,"{'neg': 0.322, 'neu': 0.511, 'pos': 0.168, 'co..."


# 7. insights 
- Use the resulting sentiment scores to gain insights into the sentiment surrounding each stock. For example, you could compare the sentiment scores of different stocks to see which ones have the most positive or negative sentiment.

In [63]:
com_sorted_score.head()

,score_compound,ticker,com_date,scores
1740,0.9923,"(CA,)",2022-12-11,"{'neg': 0.081, 'neu': 0.691, 'pos': 0.229, 'co..."
425,0.9806,"(US, ECB)",2022-12-11,"{'neg': 0.101, 'neu': 0.65, 'pos': 0.25, 'comp..."
862,0.9709,"(US, VA, GOTO)",2022-12-11,"{'neg': 0.044, 'neu': 0.711, 'pos': 0.245, 'co..."
1472,0.9486,"(TBD, TV)",2022-12-11,"{'neg': 0.104, 'neu': 0.691, 'pos': 0.205, 'co..."
183,0.9451,"(AT,)",2022-12-11,"{'neg': 0.0, 'neu': 0.481, 'pos': 0.519, 'comp..."
...,...,...,...,...
188,-0.8503,"(DD,)",2022-12-11,"{'neg': 0.402, 'neu': 0.538, 'pos': 0.06, 'com..."
682,-0.8856,"(US, GDP, RE)",2022-12-11,"{'neg': 0.202, 'neu': 0.687, 'pos': 0.111, 'co..."
359,-0.8885,"(FUCKS, REALLY, WAS, NOT, GONNA, WORK, AFTER, ...",2022-12-11,"{'neg': 0.441, 'neu': 0.392, 'pos': 0.167, 'co..."
442,-0.9196,"(SO, OP)",2022-12-11,"{'neg': 0.322, 'neu': 0.511, 'pos': 0.168, 'co..."
